In [ ]:

#Dog breed indentifier using CNN

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import cv2
import random
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
# from tensorflow.keras.callbacks import TensorBoard
import time

import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy


from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input


from tensorflow.keras.applications import vgg19
from tensorflow.keras.applications.vgg19 import preprocess_input
# from tensorflow.keras import backend as K



In [ ]:

RESULTS_DIR = '/content/drive/My Drive/results/'
IMG_SIZE = 224

BATCH_SIZE_PRE = 32
BATCH_SIZE_FINE = 16
EPOCHS_PRE = 5
EPOCHS_FINE = 5
MOMENTUM = 0.9

In [ ]:

def read_image4d_from_file(filename:str):
    image = cv2.imread(filename, cv2.IMREAD_COLOR).astype('float32')
    print("image type:", type(image))
    image_4d = np.expand_dims(image, axis=0)
    print("image_4d:", image_4d.shape)
    return image_4d

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:

def generator_from_filenames_and_labels(filenames, labels, batch_size, input_size=(299,299)):
    num_files = len(filenames)
    while 1:
        permutation = np.random.permutation(num_files)
        inputs_shuffled = filenames[permutation]
        labels_shuffled = labels[permutation]
        for i in range(0, numsamples, batch_size):
            inputs = list(map(lambda x: image.load_img(x, target_size=input_size), filenames[i:i+batch_size]))
            inputs = np.array(list(map(lambda x: image.img_to_array(x))))
            inputs = pre_process_inputs(inputs)
            yield (inputs, labels[i:i+batch_size])

In [ ]:

train_datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    rotation_range = 10,
    horizontal_flip = True,
    validation_split = 0.2)

train_pre_generator = train_datagen.flow_from_directory(
    directory = "/content/drive/My Drive/train_sub/",
    target_size = (IMG_SIZE, IMG_SIZE),
    color_mode = "rgb",
    batch_size = BATCH_SIZE_PRE,
    class_mode = "categorical")

train_fine_generator = train_datagen.flow_from_directory(
    directory = "/content/drive/My Drive/train_sub/",
    target_size = (IMG_SIZE, IMG_SIZE),
    color_mode = "rgb",
    batch_size = BATCH_SIZE_FINE,
    class_mode = "categorical")

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    directory = "/content/drive/My Drive/test_sub/",
    batch_size = BATCH_SIZE_FINE,
    class_mode = "categorical"
)

Found 723 images belonging to 5 classes.
Found 723 images belonging to 5 classes.
Found 480 images belonging to 5 classes.


In [ ]:
# dog_breed_dict = {}
# for i, breed in enumerate(dog_breed_name['breed'].unique()):
#     dog_breed_dict[breed] = i
    
# num_breeds = len(dog_breed_name['breed'].unique())
# print("num_breeds:", num_breeds)

assert len(os.listdir('/content/drive/My Drive/train_sub/')) == len(os.listdir('/content/drive/My Drive/test_sub/'))
num_breeds = len(os.listdir('/content/drive/My Drive/train_sub/'))
print("num_breeds:", num_breeds)

num_breeds: 5


In [ ]:
vgg_base_model = vgg16.VGG16(include_top=False,
                                        weights='imagenet',
                                        input_shape=(224,224,3))

x_base_vgg = vgg_base_model.output
x_avg_pooling_vgg = GlobalAveragePooling2D()(x_base_vgg)
# x_first_fc_vgg = Dense(1024, activation='relu')(x_avg_pooling_vgg)
x_last_fc_vgg = Dense(num_breeds, activation='softmax')(x_avg_pooling_vgg)

vgg_custom_model = Model(inputs=vgg_base_model.inputs, outputs=x_last_fc_vgg)


In [ ]:
#Train model - freeze body layers first
for layer in vgg_base_model.layers:
    layer.trainable = False

vgg_custom_model.compile(optimizer=SGD(momentum=MOMENTUM), loss=categorical_crossentropy, metrics=['accuracy'])

model_train_pre = vgg_custom_model.fit_generator(
    generator = train_pre_generator,
    steps_per_epoch = (train_pre_generator.n//train_pre_generator.batch_size),
    epochs = EPOCHS_PRE,
    validation_data = test_generator,
    validation_steps = (test_generator.n//test_generator.batch_size),
    verbose=1
)



<ipython-input-14-dcb8c9d73cba>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_train_pre = vgg_custom_model.fit_generator(


Epoch 1/5
22/22 [==============================] - 715s 33s/step - loss: 13.1361 - accuracy: 0.6686 - val_loss: 4.9507 - val_accuracy: 0.8604
Epoch 2/5
22/22 [==============================] - 707s 33s/step - loss: 3.6765 - accuracy: 0.9059 - val_loss: 2.6634 - val_accuracy: 0.9333
Epoch 3/5
22/22 [==============================] - 709s 33s/step - loss: 2.0649 - accuracy: 0.9291 - val_loss: 3.0765 - val_accuracy: 0.9083
Epoch 4/5
22/22 [==============================] - 706s 33s/step - loss: 0.7221 - accuracy: 0.9609 - val_loss: 3.5268 - val_accuracy: 0.9125
Epoch 5/5
22/22 [==============================] - 705s 33s/step - loss: 1.0029 - accuracy: 0.9595 - val_loss: 3.1786 - val_accuracy: 0.9187


In [17]:
# total_validate =480
# batch_size = 1

# loss, accuracy = vgg_custom_model.evaluate_generator(test_generator, total_validate//batch_size, workers=12)
# print("Test: accuracy = %f  ;  loss = %f " % (accuracy, loss))

In [18]:
vgg_custom_model.save(os.path.join(RESULTS_DIR, 'model_pre_vgg1.h5'))

In [ ]:
vgg_custom_model.save(os.path.join(RESULTS_DIR, 'model_pre_2vgg.h5'))


In [ ]:
#Train model - train all layers
for layer in vgg_base_model.layers:
    layer.trainable = True
    
vgg_custom_model.compile(optimizer=SGD(momentum=0.9), loss=categorical_crossentropy, metrics=['accuracy'] )

model_train_fine = vgg_custom_model.fit_generator(
    generator = train_fine_generator,
    steps_per_epoch = (train_fine_generator.n//train_fine_generator.batch_size),
    epochs = 5,
    validation_data = test_generator,
    validation_steps = (test_generator.n//test_generator.batch_size),
    verbose = 1
)

vgg_custom_model.save(os.path.join(RESULTS_DIR, 'model_fine.h5'))

<ipython-input-19-0821778bfc9f>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_train_fine = vgg_custom_model.fit_generator(


Epoch 1/5
45/45 [==============================] - 1735s 39s/step - loss: nan - accuracy: 0.2390 - val_loss: nan - val_accuracy: 0.2146
Epoch 2/5
45/45 [==============================] - 1721s 38s/step - loss: nan - accuracy: 0.2207 - val_loss: nan - val_accuracy: 0.2146
Epoch 3/5
45/45 [==============================] - 1691s 38s/step - loss: nan - accuracy: 0.2221 - val_loss: nan - val_accuracy: 0.2146
Epoch 4/5
45/45 [==============================] - 1707s 38s/step - loss: nan - accuracy: 0.2221 - val_loss: nan - val_accuracy: 0.2146
Epoch 5/5
45/45 [==============================] - ETA: 0s - loss: nan - accuracy: 0.2178 

In [ ]:
total_validate =480
batch_size = 1

loss, accuracy = vgg_custom_model.evaluate_generator(test_generator, total_validate//batch_size, workers=12)
print("Test: accuracy = %f  ;  loss = %f " % (accuracy, loss))

<ipython-input-21-0e73abc79267>:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss, accuracy = vgg_custom_model.evaluate_generator(test_generator, total_validate//batch_size, workers=12)


In [20]:
vgg_custom_model.save(os.path.join(RESULTS_DIR, 'model_vggfine.h5'))

In [ ]:
import sklearn.metrics
def f1_score(y_true, y_pred, average='macro'):
    """Calculate the F1 score for the given predictions and labels.
    Args:
        y_true (np.ndarray): Array of true labels.
        y_pred (np.ndarray): Array of predicted labels.
        average (str): Type of averaging to use. Options are 'micro', 'macro', and 'weighted'.
    Returns:
        float: The F1 score.
    """
    return sklearn.metrics.f1_score(y_true, y_pred, average=average)

In [ ]:
# from keras.models import load_model
# model = load_model('myModel.h5')

In [ ]:
# LOAD MODEL

from keras.models import load_model
# model = load_model('myvgg16Model.h5')

model = load_model(os.path.join(RESULTS_DIR, 'model_pre_2vgg.h5'), custom_objects={'f1_score': f1_score})


In [ ]:
# EVALUATE ON TEST SET
test_generator.reset()
pred = model.predict_generator(test_generator, steps=(test_generator.n//test_generator.batch_size), verbose=1)
# pred_bool = (pred > 0.5)
test_labels = test_generator.classes
test_labels = test_labels[:len(pred)]

# print('Accuracy:', accuracy(test_labels, pred_bool))
# print('Precision:', precision(test_labels, pred_bool))
# print('Recall:', recall(test_labels, pred_bool))
# print('AUC:', auc(test_labels, pred_bool))

<ipython-input-64-1def4af94d4d>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model.predict_generator(test_generator, steps=(test_generator.n//test_generator.batch_size), verbose=1)


 7/30 [======>.......................] - ETA: 4:22

In [ ]:
print(pred)

[[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 ...
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]


In [ ]:
# calculate f1 score for test labels and predictions
f1_score(test_labels, pred_bool, average='macro')

ValueError: ignored

In [ ]:
# calculate f1 score for test labels and predictions
f1_score(test_labels, predictions, average='macro')

def f1_score(y_true, y_pred, average='macro'):
    """Calculate the F1 score for the given predictions and labels.
    Args:
        y_true (np.ndarray): Array of true labels.
        y_pred (np.ndarray): Array of predicted labels.
        average (str): Type of averaging to use. Options are 'micro', 'macro', and 'weighted'.
    Returns:
        float: The F1 score.
    """
    return sklearn.metrics.f1_score(y_true, y_pred, average=average)

InvalidArgumentError: ignored

In [ ]:



# EVALUATE ON TRAIN SET
train_generator.reset()
pred = model.predict_generator(train_generator, steps=(train_generator.n//train_generator.batch_size), verbose=1)
pred_bool = (pred > 0.5)
train_labels = train_generator.classes
train_labels = train_labels[:len(pred_bool)]
print('F1 score:', f1_score(train_labels, pred_bool))
print('Accuracy:', accuracy(train_labels, pred_bool))
print('Precision:', precision(train_labels, pred_bool))
print('Recall:', recall(train_labels, pred_bool))
print('AUC:', auc(train_labels, pred_bool))


<function f1_score at 0x7f0bf34cb430>
